In [18]:
import os
import zipfile

import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.stats.mstats import winsorize

from Constant import Constants as const
from OrganizeData.step02_merge_all_financial_data import sort_csmar_data

In [38]:
ldx_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20240905_ldy_china_data.pkl')).rename(columns={'code': const.TICKER, 'year': const.YEAR})
er_guarantee_df: DataFrame = pd.read_csv(os.path.join(r'D:\Onedrive\Projects\CapitalControl\data\KarXiong', 'df4.csv'), usecols=['Symbol', 'Year', 'TotalLoan', 'NumGuarantee']).rename(columns={"Symbol": const.TICKER, 'Year': const.YEAR})


In [40]:
for key in ['bsize', 'totalassets', 'firmage', 'fixedassets', 'listage']:
    ldx_df[f'ln_{key}'] = np.log(ldx_df[key] + 2)
    
ldx_df.sort_values(by=[const.TICKER, const.YEAR], ascending=True, inplace=True)
ldx_df['lagged_at'] = ldx_df.groupby(const.TICKER)['totalassets'].shift(1)
ldx_df['sale_diff'] = ldx_df.groupby(const.TICKER)['sales'].diff(1)
ldx_df['sale_growth'] = ldx_df['sale_diff'] / ldx_df['lagged_at']
ldx_df['fix_at'] = ldx_df['fixedassets'] / ldx_df['lagged_at']
ldx_df['salecost_at'] = ldx_df['cost'] / ldx_df['lagged_at']
ldx_df['rev_at'] = ldx_df['revenue'] / ldx_df['lagged_at']

ldx_df.replace([np.inf, -np.inf], np.nan, inplace=True)

for key in ['zscore', 'tbq3', 'mb', 'incometaxtate', 'tbq2', 'roa', 'growth', 'tbq1', 'lev', 'tbq4', 'sale_growth', 'fix_at', 'salecost_at', 'rev_at']:
    ldx_df.loc[ldx_df[key].notnull(), key] = winsorize(ldx_df[key].dropna(), limits=(0.01, 0.01))

In [45]:
reg_df: DataFrame = ldx_df.merge(er_guarantee_df, on=[const.TICKER, const.YEAR], how='left')
reg_df.loc[:, 'Post2017'] = (reg_df[const.YEAR] >= 2017).astype(int)

# Filter the DataFrame to include only the years 2014 to 2017
df5_filtered = er_guarantee_df[(er_guarantee_df[const.YEAR] >= 2014) & (er_guarantee_df[const.YEAR] <= 2017)]

# Create a new column "er_foreign_gua" and set it to 0 initially
reg_df['er_foreign_gua'] = 0

# Identify Symbols with at least one "NumGuarantee" greater than 0 between 2014 and 2017
symbols_with_gua = df5_filtered[df5_filtered['NumGuarantee'] > 0][const.TICKER].unique()

reg_df.loc[reg_df[const.TICKER].isin(symbols_with_gua), 'er_foreign_gua'] = 1
reg_df['has_guarantee'] = 0
reg_df.loc[reg_df['NumGuarantee'] > 0, 'has_guarantee'] = 1

In [46]:
reg_df.to_pickle(os.path.join(const.TEMP_PATH, '20240908_temp_base_reg_data.pkl'))

# Construct some financial data from CSMAR

In [19]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '财务指标文件.zip'), 'r') as zip_ref:
    with zip_ref.open('CSR_Finidx.csv') as csv_file:
        finidx_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Stkcd', 'Accper', 'Outcap', 'Surplus', 'D610000', 'B150101', 'B140204', 'B140101', 'B120101', 'A100000', 'A200000']).rename(columns={'D610000': 'OCF', 'B150101': 'NetIncome', 'B140204': 'TXPD', 'B140101': 'EarningBI', 'B120101': 'OperatingRevenue', 'A100000': 'TotalAssets', 'A200000': 'TotalLiabilities'})
        finidx_df: DataFrame = sort_csmar_data(finidx_df)

C:\Users\wangy\AppData\Local\Temp\ipykernel_27200\218993644.py:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  gov_grant_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Stkcd', 'Accper', 'Item', 'Amount']).rename(


ValueError: Usecols do not match columns, columns expected but not found: ['RDInvest', 'RDSpendSumRatio', 'RDPersonRatio', 'RDInvestNetprofitRatio', 'RDInvestRatio', 'RDSpendSum', 'RDExpenses', 'RDPerson']

In [24]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '研发投入情况表.zip'), 'r') as zip_ref:
    with zip_ref.open('PT_LCRDSPENDING.csv') as csv_file:
        rd_spending_df: DataFrame = pd.read_csv(csv_file, usecols=['Symbol', 'EndDate', 'RDPerson', 'RDPersonRatio', 'RDSpendSum', 'RDSpendSumRatio', 'RDExpenses', 'RDInvest', 'RDInvestRatio', 'RDInvestNetprofitRatio'], dtype={'EndDate': 'str'})
        rd_spending_df['EndDate'] = pd.to_datetime(rd_spending_df['EndDate'], format='%Y-%m-%d', errors='coerce')
        rd_spending_df.dropna(subset=['EndDate', 'Symbol'], how='any', inplace=True)
        rd_spending_df[const.TICKER] = rd_spending_df['Symbol'].astype(int)
        rd_spending_df[const.YEAR] = rd_spending_df['EndDate'].dt.year
        for key in ['RDPerson', 'RDPersonRatio', 'RDSpendSum', 'RDSpendSumRatio', 'RDExpenses', 'RDInvest', 'RDInvestRatio', 'RDInvestNetprofitRatio']:
            rd_spending_df[key] = rd_spending_df[key].astype(np.float64)
        rd_spend_df1: DataFrame = rd_spending_df.groupby([const.TICKER, const.YEAR])[['RDPerson', 'RDSpendSum', 'RDExpenses', 'RDInvest']].sum()
        rd_spend_df2: DataFrame = rd_spending_df.groupby([const.TICKER, const.YEAR])[['RDPersonRatio', 'RDSpendSumRatio', 'RDInvestRatio', 'RDInvestNetprofitRatio']].mean()
        rd_spend_df: DataFrame = rd_spend_df1.merge(rd_spend_df2, left_index=True, right_index=True, how='outer').reset_index(drop=False)
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '政府补助.zip'), 'r') as zip_ref:
    with zip_ref.open('PT_LCGovGrants.csv') as csv_file:
        gov_grant_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Stkcd', 'Accper', 'Item', 'Amount']).rename(
            columns={'Stkcd': const.TICKER})
        gov_grant_df = gov_grant_df[gov_grant_df['Item'] == '合计']
        gov_grant_df['Accper'] = pd.to_datetime(gov_grant_df['Accper'], format='%Y-%m-%d', errors='coerce')
        gov_grant_df[const.YEAR] = gov_grant_df['Accper'].dt.year
        gov_grant_df['GovGrantAmount'] = gov_grant_df['Amount'].astype(float)
        gov_grant_df.dropna(subset=['GovGrantAmount'], inplace=True)
        gov_grant_df = gov_grant_df[gov_grant_df['GovGrantAmount'] > 0]
        gov_grant_df2: DataFrame = gov_grant_df.groupby([const.TICKER, const.YEAR])['GovGrantAmount'].sum().reset_index(drop=False)
        gov_grant_df2['lnGovGrantAmount'] = gov_grant_df2['GovGrantAmount'].apply(np.log)

C:\Users\wangy\AppData\Local\Temp\ipykernel_27200\2889229940.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  gov_grant_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Stkcd', 'Accper', 'Item', 'Amount']).rename(


In [26]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '财务指标.zip'), 'r') as zip_ref:
    with zip_ref.open('BDT_FinIndex.csv') as csv_file:
        finindex_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Symbol', 'EndDate', 'TotalAssets', 'TotalLiabilities', 'OperatingRevenue', 'IncomeTaxTate', 'TaxBearing', 'BankLoanRatio'])
        finindex_df[const.TICKER] = finindex_df['Symbol'].astype(int)
        finindex_df['EndDate'] = pd.to_datetime(finindex_df['EndDate'], format='%Y-%m-%d', errors='coerce')
        finindex_df[const.YEAR] = finindex_df['EndDate'].dt.year
        finindex_df.drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last', inplace=True)
        finindex_df.drop(['Symbol', 'EndDate'], axis=1, inplace=True)
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '融资约束—WW指数.zip'), 'r') as zip_ref:
    with zip_ref.open('BDT_FinConstWW.csv') as csv_file:
        fcww_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Symbol', 'Enddate', 'WW', 'OperatingRevenueGrowth'])
        fcww_df[const.TICKER] = fcww_df['Symbol'].astype(int)
        fcww_df['Enddate'] = pd.to_datetime(fcww_df['Enddate'], format='%Y-%m-%d', errors='coerce')
        fcww_df[const.YEAR] = fcww_df['Enddate'].dt.year
        fcww_df.drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last', inplace=True)
        fcww_df.drop(['Symbol', 'Enddate'], axis=1, inplace=True)
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '融资约束—SA指数.zip'), 'r') as zip_ref:
    with zip_ref.open('BDT_FinConstSA.csv') as csv_file:
        fcsa_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Symbol', 'Enddate', 'SA'])
        fcsa_df[const.TICKER] = fcsa_df['Symbol'].astype(int)
        fcsa_df['Enddate'] = pd.to_datetime(fcsa_df['Enddate'], format='%Y-%m-%d', errors='coerce')
        fcsa_df[const.YEAR] = fcsa_df['Enddate'].dt.year
        fcsa_df.drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last', inplace=True)
        fcsa_df.drop(['Symbol', 'Enddate'], axis=1, inplace=True)
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '融资约束—KZ指数.zip'), 'r') as zip_ref:
    with zip_ref.open('BDT_FinConstKZ.csv') as csv_file:
        fckz_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Symbol', 'Enddate', 'KZ', 'TobinQ'])
        fckz_df[const.TICKER] = fckz_df['Symbol'].astype(int)
        fckz_df['Enddate'] = pd.to_datetime(fckz_df['Enddate'], format='%Y-%m-%d', errors='coerce')
        fckz_df[const.YEAR] = fckz_df['Enddate'].dt.year
        fckz_df.drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last', inplace=True)
        fckz_df.drop(['Symbol', 'Enddate'], axis=1, inplace=True)
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '融资约束—FC指数.zip'), 'r') as zip_ref:
    with zip_ref.open('BDT_FinConstFC.csv') as csv_file:
        fcfc_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip', usecols=['Symbol', 'Enddate', 'FC', 'EBIT'])
        fcfc_df[const.TICKER] = fcfc_df['Symbol'].astype(int)
        fcfc_df['Enddate'] = pd.to_datetime(fcfc_df['Enddate'], format='%Y-%m-%d', errors='coerce')
        fcfc_df[const.YEAR] = fcfc_df['Enddate'].dt.year
        fcfc_df.drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last', inplace=True)
        fcfc_df.drop(['Symbol', 'Enddate'], axis=1, inplace=True)


In [34]:
csmar_data_df: DataFrame = finidx_df.merge(rd_spend_df, on=[const.TICKER, const.YEAR], how='outer').merge(gov_grant_df2, on=[const.TICKER, const.YEAR], how='outer').merge(finindex_df, on=[const.TICKER, const.YEAR], how='outer').merge(fcww_df, on=[const.TICKER, const.YEAR], how='outer').merge(fckz_df, on=[const.TICKER, const.YEAR], how='outer').merge(fcfc_df, on=[const.TICKER, const.YEAR], how='outer').merge(fcsa_df, on=[const.TICKER, const.YEAR], how='outer')

In [35]:
drop_keys = list()
for key in ['TotalAssets_x', 'TotalLiabilities_x', 'OperatingRevenue_x']:
    drop_keys.append(key)
    csmar_data_df.loc[:, key[:-2]] = csmar_data_df.loc[:, key].fillna(csmar_data_df[key.replace('_x', '_y')])
    drop_keys.append(key.replace('_x', '_y'))
    
csmar_data_df.drop(drop_keys, axis=1, inplace=True)
csmar_data_df.to_pickle(os.path.join(const.TEMP_PATH, '20240908_csmar_temp_data.pkl'))
csmar_data_df = csmar_data_df[csmar_data_df[const.YEAR] > 2010].copy()
csmar_data_df.shape

(56771, 31)

In [47]:
csmar_data_df.describe()

,tic,EarningBI,TXPD,NetIncome,OCF,Surplus,Outcap,year,RDPerson,RDSpendSum,...,OperatingRevenueGrowth,WW,TobinQ,KZ,EBIT,FC,SA,TotalAssets,TotalLiabilities,OperatingRevenue
count,56771.000000,4.872300e+04,4.844800e+04,4.872300e+04,4.871400e+04,4.799900e+04,4.865600e+04,56771.000000,47887.000000,4.788700e+04,...,46443.000000,40072.000000,44085.000000,44085.000000,4.408600e+04,44086.000000,46501.000000,4.873400e+04,4.873400e+04,4.819700e+04
mean,350597.883233,1.232104e+09,2.502324e+08,9.832745e+08,1.897705e+09,9.513026e+08,8.367315e+08,2017.868348,447.083697,2.176981e+08,...,3.658458,-1.161971,2.167367,0.968429,8.260854e+08,0.511121,-3.828629,6.408686e+10,5.357359e+10,1.893662e+09
std,281099.899750,1.147531e+10,2.292300e+09,9.274405e+09,2.733173e+10,7.252134e+09,6.239530e+09,3.582191,4211.965467,1.126431e+09,...,628.534677,23.682564,5.807248,2.844719,5.129697e+09,0.290494,0.323962,8.851602e+11,8.120213e+11,9.150732e+10
min,1.000000,-7.130188e+10,-1.075500e+10,-6.874256e+10,-5.880090e+11,-6.517450e+10,-7.556230e+06,2011.000000,0.000000,0.000000e+00,...,-2.683831,-4712.153159,0.608519,-13.102681,-6.517450e+10,0.000000,-5.980174,3.083701e+06,-2.033024e+06,-1.961849e+13
25%,2604.000000,3.726370e+07,5.262122e+06,3.050691e+07,1.098959e+07,4.650048e+07,3.581483e+07,2015.000000,0.000000,2.052111e+07,...,-0.045281,-1.066531,1.234627,-0.608559,4.922063e+07,0.255410,-4.018300,1.550913e+09,4.238035e+08,1.660825e+08
50%,300815.000000,1.285760e+08,1.991775e+07,1.072359e+08,1.170276e+08,1.484556e+08,1.159738e+08,2018.000000,118.000000,4.822820e+07,...,0.089951,-1.015326,1.586550,1.236070,1.544249e+08,0.546484,-3.836862,3.439027e+09,1.317950e+09,3.955523e+08
75%,601098.000000,4.237481e+08,7.138277e+07,3.534578e+08,4.501003e+08,4.726224e+08,3.483689e+08,2021.000000,332.000000,1.227450e+08,...,0.248646,-0.968374,2.277610,2.759846,4.745018e+08,0.767430,-3.656814,9.279478e+09,4.646315e+09,1.079574e+09
max,900957.000000,4.248990e+11,8.551500e+10,3.651160e+11,1.825282e+12,3.632350e+11,3.308610e+11,2023.000000,836001.000000,7.383900e+10,...,134607.058410,0.250704,729.629281,15.431583,2.555680e+11,0.998306,-0.287821,4.469708e+13,4.092049e+13,1.627613e+12
